# Creando un proyecto con las APIs de Google

1. Crear un nuevo proyecto en la consola: console.google.com
    1. En este caso se llama `gworskpace-ifts18-helper`
2. Activar las APIs que necesitás usar en **API y servicios > API y servicios habilitados**:
    1. En este caso: `Admin SDK API` en https://console.cloud.google.com/apis/library/admin.googleapis.com?authuser=0&orgonly=true&project=ambient-tuner-351923&supportedpurview=organizationId
3. Crear credenciales en **API y servicios > Credenciales**::
    1. Tipo de credencial: *Cuentas de Servicio*
    2. Elegir un nombre: `GWorkspace IFTS18 Helper`
    3. Automáticamente se crea un ID (lo podés cambiar, pero tiene sentido?): `gworkspace-ifts18-helper@fast-lattice-351622.iam.gserviceaccount.com`
    4. Descripción: "Helper para conectarse a la consola de Admin para administrar usuarios y grupos."
4. Acceso al proyecto:
    1. **Propietario**
5. En **IAM y administración > Cuentas de servicio** seleccionar la Service Account creada anteriormente e ir al tab de **Claves**:
    1. Ir a **Agregar Clave > Crear clave nueva**. Seleccionar **JSON** y descargar.
    2. Copiar el ID de la Service Account en el tab de **Detalles** (algo tipo 117773917404707229602)
6. Activar la delegación de dominio:
    1. Ir a https://admin.google.com/ac/owl/domainwidedelegation
    2. Copiar el ID y poner el SCOPE: `https://www.googleapis.com/auth/admin.directory.user`

Ahora tenemos nuestro proyecto en Google y tenemos las credenciales para autenticarnos, podemos empezar a codear.

# Instalando el ambiente

1. Create un nuevo virtualenv, activalo y actualizá `pip`: `python -m venv venv && source venv/bin/activate.fish && python -m pip install --upgrade pip setuptools`
2. Instalá *Jupyter Lab*: `python -m pip install jupyterlab`
3. Instalá las "Google libs": `python -m pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib`
4. Creá un link simbólico que se llame `credentials.json` al JSON que tiene tus credenciales bajadas

# Docs

- [Using OAuth 2.0 to Access Google APIs](https://developers.google.com/identity/protocols/oauth2)
    - [Scopes](https://developers.google.com/identity/protocols/oauth2/scopes) 
- [Python Quickstart Admin SDK](https://developers.google.com/admin-sdk/directory/v1/quickstart/python)
- [Admin SDK REST Reference](https://developers.google.com/admin-sdk/directory/reference/rest)

# Logueate con la API

In [1]:
import json

from google.oauth2 import service_account
from googleapiclient.discovery import build

SERVICE_ACCOUNT_FILE = 'credentials/credentials3.json'
SCOPES = [
    'https://www.googleapis.com/auth/admin.directory.user',
    'https://www.googleapis.com/auth/gmail.compose', 
]


service_account_info = json.load(open(SERVICE_ACCOUNT_FILE))

credentials = service_account.Credentials.from_service_account_info(
        service_account_info, scopes=SCOPES)

SUBJECT = 'colomboleandro@ifts18.edu.ar'

delegated_creds = credentials.with_subject(SUBJECT)

admin_service = build('admin', 'directory_v1', credentials=delegated_creds)

# Call the Admin SDK Users API
results = admin_service.users().list(domain='alu.ifts18.edu.ar', maxResults=100,
                                   orderBy='email').execute()
users = results.get('users', [])

if not users:
    print('No users found.')
else:
    print(f'{len(users)} users:')
    for user in users:
        print(f'{user["primaryEmail"]}')

67 users:
agustin.sabia@alu.ifts18.edu.ar
alan.baral@alu.ifts18.edu.ar
alan.rolandria@alu.ifts18.edu.ar
alberto.campagna@alu.ifts18.edu.ar
augusto.luna@alu.ifts18.edu.ar
camila.quispe@alu.ifts18.edu.ar
carlos.choque@alu.ifts18.edu.ar
carolina.bernachea@alu.ifts18.edu.ar
dayana.bernal@alu.ifts18.edu.ar
diego.benitez@alu.ifts18.edu.ar
diego.gimenez@alu.ifts18.edu.ar
eliana.aguilera@alu.ifts18.edu.ar
elvio.vaca@alu.ifts18.edu.ar
emanuel.odstrcil@alu.ifts18.edu.ar
emiliano.palacios@alu.ifts18.edu.ar
emir.gonzalez@alu.ifts18.edu.ar
enzo.zeballos@alu.ifts18.edu.ar
erick.krause@alu.ifts18.edu.ar
fernanda.martinelli@alu.ifts18.edu.ar
fernando.larosa@alu.ifts18.edu.ar
fernando.warno@alu.ifts18.edu.ar
flor.culqui@alu.ifts18.edu.ar
florencia.valdez@alu.ifts18.edu.ar
franco.alvarez@alu.ifts18.edu.ar
franco.cesanelli@alu.ifts18.edu.ar
franco.pirovano@alu.ifts18.edu.ar
gabriel.paez@alu.ifts18.edu.ar
gaston.cejas@alu.ifts18.edu.ar
gonzalo.roget@alu.ifts18.edu.ar
greta.liverotti@alu.ifts18.edu.ar
gust

In [2]:
import string
import secrets

def generate_password(length=10, digits=3):
    """Generate a ten-character alphanumeric password with at least one lowercase character, at least one uppercase character, and at least three digits.
    
    Example from Python Docs: https://docs.python.org/3/library/secrets.html
    """
    alphabet = string.ascii_letters + string.digits
    while True:
        password = ''.join(secrets.choice(alphabet) for i in range(10))
        if (any(c.islower() for c in password)
                and any(c.isupper() for c in password)
                and sum(c.isdigit() for c in password) >= 3):
            break
    return password

In [3]:
def create_student_body(
    student_firstname,
    student_lastname,
    student_email,
    password=None,
    email_domain="alu.ifts18.edu.ar",
    orgUnitPath="/Alumnos",
    **kwargs,
):
    if password is None:
        # TODO: read kwargs and modify generate_password accordingly
        generated_password = generate_password()
    else:
        generated_password = password

    user = {
        "emails": [
            student_email,
        ], 
        "name": {
            "familyName": student_lastname.strip(),
            "givenName": student_firstname.strip(),
            "fullName": f"{student_firstname} {student_lastname.strip()}", 
        },
        "changePasswordAtNextLogin": True,
        "primaryEmail": f"{student_firstname.split()[0].lower().strip()}.{student_lastname.strip().lower()}@{email_domain}",
        "password": generated_password,
        "orgUnitPath": orgUnitPath,
    }
    return user

In [4]:
import csv

STUDENTS_DATA_FILE = "assets/example.csv"

def read_students_csv(filename):
    "Reads Students CSV and returns a list of dicts with the students data"
    students = []
    with open(filename) as csvfile:
        studentreader = csv.DictReader(csvfile)
        for student in studentreader:
            students.append(student)
    return students

In [5]:
students = read_students_csv(STUDENTS_DATA_FILE)

In [6]:
students

[{'N°': '0',
  'APELLIDO': 'Blanca',
  'NOMBRE': 'Baltasar',
  'DNI': '050-32-9548',
  'MAIL': 'luisinafigueras@example.net'},
 {'N°': '1',
  'APELLIDO': 'Espinosa',
  'NOMBRE': 'Amaro',
  'DNI': '384-34-6575',
  'MAIL': 'toni03@example.net'},
 {'N°': '2',
  'APELLIDO': 'Manuel',
  'NOMBRE': 'Nicanor',
  'DNI': '562-79-2386',
  'MAIL': 'jose-luisromeu@example.org'},
 {'N°': '3',
  'APELLIDO': 'Rosa',
  'NOMBRE': 'Miguel Ángel',
  'DNI': '230-01-3521',
  'MAIL': 'tapiaestefania@example.com'},
 {'N°': '4',
  'APELLIDO': 'Lorenzo',
  'NOMBRE': 'Natalio',
  'DNI': '108-10-2913',
  'MAIL': 'fuentesjoaquin@example.net'},
 {'N°': '5',
  'APELLIDO': 'Benavides',
  'NOMBRE': 'Valentín',
  'DNI': '458-44-7681',
  'MAIL': 'marta09@example.net'},
 {'N°': '6',
  'APELLIDO': 'Adadia',
  'NOMBRE': 'Azahara',
  'DNI': '974-20-7728',
  'MAIL': 'varco@example.com'},
 {'N°': '7',
  'APELLIDO': 'Roman',
  'NOMBRE': 'Roque',
  'DNI': '621-02-0831',
  'MAIL': 'fcarnero@example.net'},
 {'N°': '8',
  'APELLID

In [7]:
user = create_student_body(students[0]["NOMBRE"], students[0]["APELLIDO"], students[0]["MAIL"])
user

{'emails': ['luisinafigueras@example.net'],
 'name': {'familyName': 'Blanca',
  'givenName': 'Baltasar',
  'fullName': 'Baltasar Blanca'},
 'changePasswordAtNextLogin': True,
 'primaryEmail': 'baltasar.blanca@alu.ifts18.edu.ar',
 'password': 'mE4ic93Afm',
 'orgUnitPath': '/Alumnos'}

In [8]:
result = admin_service.users().insert(body=user).execute()
result

{'kind': 'admin#directory#user',
 'id': '109568668775128004766',
 'etag': '"Ap9BCC4uRt3h_SrDO0G_EX9zYVZwuEfjT_jX812ihkE/24cFv6aTti5xQvewiRQId5YmoP4"',
 'primaryEmail': 'baltasar.blanca@alu.ifts18.edu.ar',
 'name': {'givenName': 'Baltasar',
  'familyName': 'Blanca',
  'fullName': 'Baltasar Blanca'},
 'isAdmin': False,
 'isDelegatedAdmin': False,
 'creationTime': '2022-06-23T13:27:35.000Z',
 'changePasswordAtNextLogin': True,
 'customerId': 'C02zkxcvl',
 'orgUnitPath': '/Alumnos',
 'isMailboxSetup': False}

In [9]:
import base64

from email.message import EmailMessage


SUBJECT = 'inscripciones@ifts18.edu.ar'

delegated_creds = credentials.with_subject(SUBJECT)

gmail_service = build('gmail', 'v1', credentials=delegated_creds)

message = EmailMessage()
message.set_content('This is automated mail with Reply-To header for Leandro Colombo Viña. x3')
# email headers
message['To'] = 'Leo Colombo Viña <colomboleandro@gmail.com>'
message['From'] = '"Inscripciones IFTS18" <inscripciones@ifts18.edu.ar>'
message['Reply-To'] = 'no-reply@ifts18.edu.ar'
message['Subject'] = 'Your new IFTS18 student email account is created - No Reply'

# encoded message
encoded_message = base64.urlsafe_b64encode(message.as_bytes()).decode()

create_message = {
    'raw': encoded_message,
}

In [10]:
message.as_string()

'Content-Type: text/plain; charset="utf-8"\nContent-Transfer-Encoding: base64\nMIME-Version: 1.0\nTo: Leo Colombo =?utf-8?q?Vi=C3=B1a?= <colomboleandro@gmail.com>\nFrom: "Inscripciones IFTS18" <inscripciones@ifts18.edu.ar>\nReply-To: no-reply@ifts18.edu.ar\nSubject: Your new IFTS18 student email account is created - No Reply\n\nVGhpcyBpcyBhdXRvbWF0ZWQgbWFpbCB3aXRoIFJlcGx5LVRvIGhlYWRlciBmb3IgTGVhbmRybyBD\nb2xvbWJvIFZpw7FhLiB4Mwo=\n'

In [11]:
# pylint: disable=E1101
send_message = gmail_service.users().messages().send(
    userId="me",
    body=create_message
).execute()

In [12]:
send_message

{'id': '18190bf6a0009a69',
 'threadId': '18190bf6a0009a69',
 'labelIds': ['SENT']}

# Con archivos Adjuntos

## Imágenes (JPG)

In [217]:
import base64
import mimetypes
from email.message import EmailMessage


SUBJECT = 'inscripciones@ifts18.edu.ar'

delegated_creds = credentials.with_subject(SUBJECT)

gmail_service = build('gmail', 'v1', credentials=delegated_creds)

attachment_filename = 'assets/doc.pdf'
maintype, subtype = mimetypes.guess_type(attachment_filename)[0].split('/')
print(f"MIME Type detected: {maintype}/{subtype}")

message = EmailMessage()
message.set_content(f'This is automated mail with Reply-To header for Leandro Colombo Viña with a {maintype}/{subtype} attached!')
# email headers
message['To'] = 'Leo Colombo Viña <colomboleandro@gmail.com>'
message['From'] = '"Inscripciones IFTS18" <inscripciones@ifts18.edu.ar>'
message['Reply-To'] = 'no-reply@ifts18.edu.ar'
message['Subject'] = f'Your new IFTS18 student email account is created - No Reply - {maintype}/{subtype}'

with open(attachment_filename, 'rb') as fp:
    attachment_data = fp.read()
message.add_attachment(attachment_data, maintype=maintype, subtype=subtype)

# encoded message
encoded_message = base64.urlsafe_b64encode(message.as_bytes()).decode()

create_message = {
    'raw': encoded_message,
}

MIME Type detected: application/pdf


In [218]:
# pylint: disable=E1101
send_message = gmail_service.users().messages().send(
    userId="me",
    body=create_message
).execute()

# Probando Logs

In [13]:
LOGFILE = 'accounts.json'

with open(LOGFILE) as f:
    data = json.load(f)
    

In [14]:
mails = [student['user']['primaryEmail'] for student in data]

In [16]:
from faker import Faker

fake = Faker('es')

students = []
examples = 10

for i in range(examples):
    students.append(
        {
            "N°": i,
            "APELLIDO": fake.last_name(),
            "NOMBRE": fake.first_name(),
            "DNI": fake.ssn(),
            "MAIL": fake.email(),
        }
    )
print(students)

[{'N°': 0, 'APELLIDO': 'Requena', 'NOMBRE': 'Adelia', 'DNI': '499-87-1823', 'MAIL': 'danielcarranza@example.org'}, {'N°': 1, 'APELLIDO': 'Garzón', 'NOMBRE': 'Gonzalo', 'DNI': '621-36-9759', 'MAIL': 'acevedotrini@example.net'}, {'N°': 2, 'APELLIDO': 'Valenciano', 'NOMBRE': 'Adelaida', 'DNI': '747-96-8324', 'MAIL': 'tellezamaro@example.net'}, {'N°': 3, 'APELLIDO': 'Méndez', 'NOMBRE': 'Espiridión', 'DNI': '216-22-9988', 'MAIL': 'vanesa18@example.org'}, {'N°': 4, 'APELLIDO': 'Céspedes', 'NOMBRE': 'Carlito', 'DNI': '975-60-9188', 'MAIL': 'igras@example.com'}, {'N°': 5, 'APELLIDO': 'Campillo', 'NOMBRE': 'Edmundo', 'DNI': '214-19-4165', 'MAIL': 'chema82@example.com'}, {'N°': 6, 'APELLIDO': 'Fuertes', 'NOMBRE': 'María Pilar', 'DNI': '173-75-9655', 'MAIL': 'sorianozaida@example.net'}, {'N°': 7, 'APELLIDO': 'Pastor', 'NOMBRE': 'Javi', 'DNI': '326-73-9796', 'MAIL': 'elorzaruth@example.net'}, {'N°': 8, 'APELLIDO': 'Morante', 'NOMBRE': 'Emigdio', 'DNI': '328-81-3787', 'MAIL': 'goicoecheapepito@exam

In [17]:
filename_path = "assets/example.csv"

with open(filename_path, "w") as fd:
        studentwriter = csv.DictWriter(fd, fieldnames=students[0].keys())
        studentwriter.writeheader()
        for student in students:
            studentwriter.writerow(student)